<a href="https://colab.research.google.com/github/JonHaz/Capstone-2-LVMH-Sales-in-Iowa/blob/main/LVMH_Capstone_Two_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LVMH Moët Hennessy Louis Vuitton, commonly known as LVMH, is a French Holding multinational corporation and conglomerate specializing in luxury goods, headquartered in Paris, France. Is brand that steadily becoming a household name, but in terms of their footprint in Iowa what can we learn from their Wine and Spirit sales within the state.

In my role as a Product Analyst I often use BigQuery and other Cloud Datawarehouses so I thought it be interesting to complete this project entirely in the cloud.

In [1]:
# I am establishing a connection to my Sandbox within BigQuery so that I can run SQL queries to interact with the data here within Google Collab
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
# Lets see what data we are working with
%%bigquery --project storied-sound-253213 
SELECT * 
FROM `bigquery-public-data.iowa_liquor_sales.sales`
LIMIT 10

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-25006100046,2020-02-04,3447,Sam's Club 6432 / Sioux City,4201 S. York St.,Sioux City,51106,POINT (-96.37082 42.43407),97,WOODBURY,1092100,Imported Distilled Spirit Specialty,434,LUXCO INC,75087,Juarez Gold DSS,12,1000,5.00,7.50,48,360.00,48.0,12.68
1,INV-25048000012,2020-02-06,4960,AJ's Liquor III,2401 A Chamberlain St,Ames,50010,POINT (-93.648959 42.021456),85,STORY,1062100,Gold Rum,434,LUXCO INC,45247,Paramount Gold Rum,12,1000,4.42,6.63,72,477.36,72.0,19.02
2,INV-30846500020,2020-10-07,2543,Hy-Vee Food Store #1 / Ottumwa,1025 North Quincy Ave,Ottumwa,52501,POINT (-92.438463 41.01558),90,Wapello,1011600,Straight Rye Whiskies,085,Brown Forman Corp.,27181,Jack Daniels Tennessee Rye,12,750,15.57,23.36,24,560.64,18.0,4.75
3,INV-30951200042,2020-10-12,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.773557 41.561197),77,POLK,1011600,Straight Rye Whiskies,255,Infinium Spirits,27125,Templeton 6YR Rye,6,750,22.75,34.13,30,1023.90,22.5,5.94
4,INV-30910900037,2020-10-09,3944,Sam's Club 4973 / Dubuque,4400 Asbury Rd,Dubuque,52002,POINT (-90.737821 42.515289),31,DUBUQUE,1011600,Straight Rye Whiskies,255,Infinium Spirits,27102,Templeton 4YR Rye,6,750,18.09,27.14,120,3256.80,90.0,23.77
5,INV-30953700158,2020-10-12,2573,Hy-Vee Food Store / Muscatine,2400 2nd Ave,Muscatine,52761,POINT (-91.035138 41.451349),70,MUSCATINE,1091300,Neutral Grain Spirits Flavored,346,OLE SMOKY DISTILLERY LLC,86739,Ole Smoky Apple Pie Moonshine 70prf Mini,8,50,8.75,13.13,4,52.52,0.2,0.05
6,S29656700013,2015-12-15,2508,Hy-Vee Food Store #1 / Cedar Rapids,"1843 JOHNSON AVENUE, N.W.",CEDAR RAPIDS,52405,POINT (-91.697941 41.97447),57,Linn,1081340,ROOT BEER SCHNAPPS,65,Jim Beam Brands,82827,Dekuyper Rootbeer,12,1000,7.62,11.43,4,45.72,4.0,1.06
7,INV-23092300034,2019-11-07,5676,Family Pantry,4538 Lower Beaver Rd,Des Moines,50310,POINT (-93.665871 41.638673),77,POLK,None,None,055,SAZERAC NORTH AMERICA,84197,99 Pineapple Mini,10,600,6.00,9.00,1,9.00,0.6,0.15
8,S15040800017,2013-10-09,3973,Mmdg Spirits / Ames,126A WELCH AVE,AMES,50014,POINT (-93.650131 42.021789),85,Story,1081312,BUTTERSCOTCH SCHNAPPS,65,Jim Beam Brands,82787,Dekuyper Buttershots,12,1000,7.62,11.43,4,45.72,4.0,1.06
9,S26173800003,2015-06-18,5144,Sam's Club 6979 / Ankeny,4625 SE DELAWARE AVE,ANKENY,50021,POINT (-93.581273 41.691),77,Polk,1081330,PEACH SCHNAPPS,65,Jim Beam Brands,82847,Dekuyper Peachtree,12,1000,7.62,11.43,156,1783.08,156.0,41.21


In [3]:
#Seeing that I am primarily interested in the sales from LVMH for this project I am going to run a query to determine what is the vendor name

%%bigquery --project storied-sound-253213 
SELECT DISTINCT vendor_name, item_description
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE vendor_name LIKE '%HENNESSY%'

,vendor_name,item_description
0,MOET HENNESSY USA,Glenmorangie Taster Pack
1,MOET HENNESSY USA,Glenmorangie Pioneer Pack
2,MOET HENNESSY USA,Glenmorangie Lasanta
3,MOET HENNESSY USA,Glenmorangie 10YR
4,MOET HENNESSY USA,Ardbeg An Oa
...,...,...
116,MOET HENNESSY USA,Belvedere Pure w/Martini Glass
117,MOET HENNESSY USA,Hennessy Paradis Imperial
118,MOET HENNESSY USA,HA Glenmorangie Grand Vintage 1991
119,MOET HENNESSY USA,Hennessy Paradis Imperial 750ML


I recognize these brands and the vendor name looks like a match. So what I will do is load all of the data related to the vendor MOET HENNESSY USA into a Pandas DataFrame

In [5]:
#Google collab load Pandas by default but typically I would 
#import pandas as pd
# Save output in a variable `df`
%%bigquery --project storied-sound-253213 df
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE vendor_name LIKE '%HENNESSY%'

In [6]:
#Lets inspect our new dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175379 entries, 0 to 175378
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   invoice_and_item_number  175379 non-null  object 
 1   date                     175379 non-null  object 
 2   store_number             175379 non-null  object 
 3   store_name               175379 non-null  object 
 4   address                  173783 non-null  object 
 5   city                     173783 non-null  object 
 6   zip_code                 173783 non-null  object 
 7   store_location           158121 non-null  object 
 8   county_number            173783 non-null  object 
 9   county                   173783 non-null  object 
 10  category                 174978 non-null  object 
 11  category_name            174978 non-null  object 
 12  vendor_number            175379 non-null  object 
 13  vendor_name              175379 non-null  object 
 14  item

It looks like there are 175,379 entries and none of the field are null!

But it appears that not all of the columns have the correct datatype so we would want to fix this.

In [7]:
df.head()

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-30752900191,2020-10-05,2619,Hy-Vee Wine and Spirits / WDM,1725 74th St,West Des Moines,50266,POINT (-93.808855 41.598515),77,POLK,1012300,Single Malt Scotch,420,MOET HENNESSY USA,5104,Glenmorangie Nectar D'or,6,750,34.99,52.49,18,944.82,13.5,3.56
1,INV-20918000010,2019-07-30,3814,Costco Wholesale #788 / WDM,7205 Mills Civic Pkwy,West Des Moines,50266,POINT (-93.806489 41.561342),25,Dallas,1012300,Single Malt Scotch,420,MOET HENNESSY USA,5136,Glenmorangie 18YR,6,750,49.99,74.99,18,1349.82,13.5,3.56
2,INV-10472100006,2018-02-19,2190,"Central City Liquor, Inc.",1460 2ND AVE,Des Moines,50314,POINT (-93.619787 41.60566),77,POLK,1012300,Single Malt Scotch,420,MOET HENNESSY USA,990434,HA Glenmorangie Spios,6,750,49.99,74.99,30,2249.70,22.5,5.94
3,INV-07182900001,2017-09-12,2588,Hy-Vee Food and Drug #6 / Cedar Rapids,4035 Mt Vernon Rd SE,Cedar Rapids,52403,POINT (-91.60978 41.976835),57,LINN,1012300,Single Malt Scotch,420,MOET HENNESSY USA,5104,Glenmorangie Nectar D'or,6,750,35.49,53.24,36,1916.64,27.0,7.13
4,INV-06905200006,2017-08-28,4169,Super Quick 2 / Hubbell,1824 Hubbell Ave,Des Moines,50317,None,77,POLK,None,None,420,MOET HENNESSY USA,163,Hennessy VS Flask,24,200,5.99,8.99,24,215.76,4.8,1.27


In [12]:
#in order to use the pd shorthand for pandas it seems i do need to 
#import pandas as pd
#lesson learned

import pandas as pd
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175379 entries, 0 to 175378
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   invoice_and_item_number  175379 non-null  object        
 1   date                     175379 non-null  datetime64[ns]
 2   store_number             175379 non-null  object        
 3   store_name               175379 non-null  object        
 4   address                  173783 non-null  object        
 5   city                     173783 non-null  object        
 6   zip_code                 173783 non-null  object        
 7   store_location           158121 non-null  object        
 8   county_number            173783 non-null  object        
 9   county                   173783 non-null  object        
 10  category                 174978 non-null  object        
 11  category_name            174978 non-null  object        
 12  vendor_number   

Before going forward one thing i want to verify is what field can be used as an index

In [16]:
df.nunique()

invoice_and_item_number    175379
date                         1256
store_number                 1952
store_name                   2027
address                      1898
city                          397
zip_code                      434
store_location               1678
county_number                  99
county                        129
category                       10
category_name                  10
vendor_number                   1
vendor_name                     1
item_number                   120
item_description              121
pack                            9
bottle_volume_ml               11
state_bottle_cost              86
state_bottle_retail            86
bottles_sold                   91
sale_dollars                 1112
volume_sold_liters            176
volume_sold_gallons           222
dtype: int64

Okay so this tells me quite a lot about my dataframe such as

*   df['invoice_and_item_number'] are my unique values
*   There are 10 categories, 9 differenct packages and 11 bottle sizes

At a glance my data looks nice, clean and tidy.